<a href="https://colab.research.google.com/github/woojin-blaze/E-commerce-llm/blob/main/blaze_recall_saved_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 16.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64

In [ ]:
!pip install --upgrade datasets

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
#!pip install bitsandbytes==0.39.0

In [ ]:

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
#!pip install accelerate==0.21.0

In [ ]:
#!pip install -U accelerate

In [ ]:
#!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/창통설_Q조_프로젝트"

Mounted at /content/drive
data			  data_labeling.ipynb  feature_addition.ipynb		generated_text
data_generation.ipynb	  demo		       finetuned_data_evaluation.ipynb	init.ipynb
data_labeling_base.ipynb  eval		       finetuning.ipynb			results


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
'''
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    #device_map = "auto"
    device_map={"": 0}
    #device_map="cpu"
)
#device = torch.device('cpu')
#model = model.to(device)
model.config.use_cache = False
model.config.pretraining_tp = 1
'''

In [ ]:
#new_model = "/content/drive/MyDrive/ColabNotebooks/results/llama-base-format-3epoch"

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt1 = "### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.\n### Raw Source: 새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌\n### Response:"


In [ ]:
prompt2 = "### Instruction: Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be relevant to the raw source.\n### Raw Source:전신 2024형 스마트 스포츠 워치 남녀공용 심박수 모니터링 블루투스 전화 팔찌 GPS 위치 추적 방수 다기능 스포츠 모드\n### Response:2024형 전신 스마트 스포츠 워치는 남녀 모두에게 적합하며, 심박수 모니터링 기능이 탑재되어 건강 관리를 한층 더 체계적으로 도와줍니다.\
블루투스를 통한 전화 연결 기능이 있어, 언제 어디서나 손쉽게 통화가 가능하며 중요한 연락을 놓치지 않습니다.\
내장된 GPS 위치 추적 시스템으로 운동 시 이동 경로를 정확하게 기록하며, 다양한 환경에서의 활동을 지원합니다.\
방수 기능이 추가되어 수영이나 비 오는 날에도 걱정 없이 사용할 수 있습니다.\
다기능 스포츠 모드가 포함되어 있어, 사용자의 다양한 운동 요구에 맞춰 최적의 운동 데이터를 제공합니다.\
### Instruction: Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be korean and relevant to the raw source.\n### Raw Source: 창의적인 다기능 책상 정리함 사무실 학생 문구함 가정용 미니 서랍형 화장품 정리함\n### Response:"


In [ ]:
#prompt = "[INST]Given a raw source for a product, generate five detailed descriptions in Korean.[/INST]창의적인 다기능 책상 정리함 사무실 학생 문구함 가정용 미니 서랍형 화장품 정리함"
'''
input_text = f"{prompt1}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=50
    )
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Title:", response)
'''

In [ ]:
'''
input_text = f"{prompt2}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=1024,
        do_sample=True,  # Enable sampling if you want diverse results
        temperature=0.9,  # Adjust based on desired creativity
        top_p=0.95,
        top_k=50
    )
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)
'''

저장한 모델 불러와보기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
my_model_path = "/content/drive/MyDrive/창통설_Q조_프로젝트/results/llama-base-format-3epoch"
my_model = AutoModelForCausalLM.from_pretrained(my_model_path)
my_tokenizer = AutoTokenizer.from_pretrained(my_model_path)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.\n\n### Raw Source:눈길 자전거 초광폭 타이어 변속 4.0 눈길용 자전거 성인 학생용 자전거 남녀공용 마운틴바이크\n### Product Title:"
pipe = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=255)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.

### Raw Source:눈길 자전거 초광폭 타이어 변속 4.0 눈길용 자전거 성인 학생용 자전거 남녀공용 마운틴바이크
### Product Title:4.0 눈길용 자전거 초광폭 타이어 남녀공용 마운틴바이크
### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.
### Raw Source:2023 신제품 스마트 스포츠 워치 심박수 모니터링 다기능 


In [ ]:
test_prompt = "### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.\n\n### Raw Source:눈길 자전거 초광폭 타이어 변속 4.0 눈길용 자전거 성인 학생용 자전거 남녀공용 마운틴바이크\n### Product Title:"
pipe = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=200)
result = pipe(f"{test_prompt}")
print(result[0]['generated_text'])

In [ ]:
prompt1 = "Given a raw source for a product, generate a coherent and relevant product title in Korean."
raw_source = "새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌"
pipe = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=200)
result = pipe(f"{prompt}\n raw source: {raw_source}")
print(result[0]['generated_text'])

[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]
 raw source: 새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌

Product title in Korean: 2023 스마트워치 터치스크린 운동 심박수 혈압 수면추적 남녀


In [ ]:
prompt1 = "Given a raw source for a product, generate a coherent and relevant product title in Korean."
prompt2 = "Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be relevant to the raw source."
prompt3 = "Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable."
raw_source = "새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌"
pipe = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=2048)
result = pipe(f"raw source: {raw_source}\n ###Product Title: \n ###Product Description: \n ###Related Keywords: ")
print(result[0]['generated_text'])

raw source: 새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌
 ###Product Title: 
 ###Product Description: 
 ###Related Keywords: 
###Product Specifications: 
###Product Reviews: 
###Product Shipping: 
###Product Return: 
###Product Seller: 
###Product Format: 
###Product Language: 
###Product Location: 
###Product Condition: 
###Product Season: 
###Product Style: 
###Product Material: 
###Product Color: 
###Product Size: 
###Product Weight: 
###Product Brand: 
###Product Model: 
###Product Type: 
###Product Subtype: 
###Product Variant: 
###Product Features: 
###Product Includes: 
###Product Accessories: 
###Product Warranty: 
###Product Support: 
###Product Safety: 
###Product Maintenance: 
###Product Durability: 
###Product Lifespan: 
###Product Cost: 
###Product Profit: 
###Product Investment: 
###Product Risk: 
###Product Opportunity: 
###Product Growth: 
###Product Expansion: 
###Product Development: 
###Product Innovation: 
###Product Improvement: 
###Product Efficien

In [ ]:
input_text = f"{prompt}\n raw source: {raw_source}\n detailed description:"
input_ids = my_tokenizer(input_text, return_tensors="pt").input_ids

with torch.no_grad():
    output_ids = my_model.generate(
        input_ids,
        max_length=1024,
        do_sample=True,
        temperature=0.9,
        top_p=0.95,
        top_k=50
    )
response = my_tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(response)

[INST]Given a raw source for a product, generate a coherent and relevant product title in Korean.[/INST]
 raw source: 새로운 스마트워치 2023 터치 스크린 운동 모니터링 심박수 혈압 수면 추적 다기능 남녀 공용 블루투스 팔찌
 detailed description: 최신 스마트워치는 터치 스크린을 통해 사용자가 편리하게 운동 모니터링을 수행할 수 있어 건강 관리에 탁월합니다.

심박수, 혈압, 수면 등의 건강 측정 기능이 포함되어 있어 일상의 건강 상태를 체크할 수 있습니다.

블루투스 기능을 통해 스마트폰이나 태블릿에 연동되어 음악 플레이, 전화, 메시지 등의 애플리케이션을 손쉽게 접근할 수 있습니다.

남녀 누구나 공용이 가능한 디자인으로 세련되고 심플한 스타일을 제공합니다.

2023년 신제품으로 현대적인 디자인과 다기능성을 갖춘 스마트워치를 찾는 유익함을 갖추고 있습니다.

### Product Title: 2023 스마트워치 운동 심박수 혈압 수면 추적 블루투스 팔찌
### Related keywords: 스마트워치 운동 측정, 심박수 측정 시계, 혈압 측정 스마트워치, 수면 추적 스마트워치, 블루투스 스마트워치, 다기능 스마트워치
### Product Description: 스마트워치 2023은 일상의 건강 체크를 손쉽게 가능하게 해주는 최신 기술을 탑재하고 있습니다. 터치 스크린을 통해 사용자는 더욱 편리하고 실용적인 운동 모니터링을 할 수 있습니다. 심박수, 혈압, 수면 등의 건강 측정 기능이 내장되어 있어 건강 


In [ ]:
 import re

def extract_response(response):

    patterns = [
        (r"### Product Title:\s*(.*?)(?=\n###|$)", "Product Title"),
        (r"### Related Keywords:\s*(.*?)(?=\n###|$)", "Related Keywords"),
        (r"### Detailed Description:\s*(.*?)(?=\n###|$)", "Detailed Description"),
        (r"Product title in Korean:\s*(.*?)(?=\n###|$)", "Product Title"),
        (r"Product Title:\s*(.*?)(?=\n###|$)", "Product Title"),
        (r"Detailed Description\s*:\s*(.*?)(?=\n###|$)", "Detailed Description"),
        (r"Related keywords:\s*(.*?)(?=\n###|$)", "Related Keywords"),
        (r"\[INST\].*?\[/INST\]\s*(.*?)(?=\n###|$)", "General"),
        (r"<s>\[INST\].*?\[/INST\]\s*(.*?)(?=\n###|$)", "General")
    ]

    for pattern, type_ in patterns:
        match = re.search(pattern, response, re.DOTALL)
        if match:
            title = match.group(1).strip()

            if type_ == "Related Keywords":

                title = re.sub(r"[^\[\],가-힣\s\-\d]", "", title).strip()
            else:

                title = re.sub(r"<\/?s>|<\/?INST>|raw source:.*", "", title).strip()
                title = re.sub(r"[^가-힣\s\-\d,]", "", title).strip()

            if re.search(r"[가-힣]", title):
                return title

    return "No text found."



In [ ]:
base_test_path = "/content/drive/MyDrive/창통설_Q조_프로젝트/data/base0619_test_split_212.csv"

In [ ]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer


base_test = pd.read_csv(base_test_path)

import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import time



pipe_product_title = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)
pipe_detailed_description = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=1024)
pipe_search_keywords = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)


timing_info = {
    "Product Title": {"total_time": 0, "count": 0},
    "Detailed Description": {"total_time": 0, "count": 0},
    "Related Keywords": {"total_time": 0, "count": 0}
}

results = []


for index, row in base_test.iterrows():
    if index >= 51 :
      break

    prompt = row['text']
    result = None
    start_time = time.time()

    if row['type'] == "Product Title":
        result = pipe_product_title(f"{prompt}")
        timing_info["Product Title"]["count"] += 1
    elif row['type'] == "Detailed Description":
        result = pipe_detailed_description(f"{prompt}")
        timing_info["Detailed Description"]["count"] += 1
    elif row['type'] == "Related Keywords":
        result = pipe_search_keywords(f"{prompt}")
        timing_info["Related Keywords"]["count"] += 1

    end_time = time.time()
    elapsed_time = end_time - start_time
    timing_info[row['type']]["total_time"] += elapsed_time

    if result:
        generated_text = result[0]['generated_text']
        ext = extract_response(generated_text)

        print(f"Prompt: {prompt}")
        print(f"Generated Text: {generated_text}")
        print("\n" + ext)
        print("-" * 80)

        results.append({
            'prompt': prompt,
            'type': row['type'],
            'generated_text': ext,
            'total': generated_text
        })


results_df = pd.DataFrame(results)


for text_type, info in timing_info.items():
    if info['count'] > 0:
        average_time = info['total_time'] / info['count']
        print(f"Average time for {text_type}: {average_time:.2f} seconds")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prompt: ### Instruction: Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable.

### Raw Source:지능형 감지 쓰레기통 가정용 거실 주방 전동 자동 감지 방취 쓰레기통 대용량 친환경 쓰레기통
### Related Keywords:
Generated Text: ### Instruction: Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable.

### Raw Source:지능형 감지 쓰레기통 가정용 거실 주방 전동 자동 감지 방취 쓰레기통 대용량 친환경 쓰레기통
### Related Keywords: 지능형 쓰레기통, 자동 쓰레기통, 대용량 쓰레기통, 전동 쓰레기통, 거실 쓰레기통, 주방 쓰레기통, 친환경 쓰레기통

지능형 쓰레기통, 자동 쓰레기통, 대용량 쓰레기통, 전동 쓰레기통, 거실 쓰레기통, 주방 쓰레기통, 친환경 쓰레기통
--------------------------------------------------------------------------------
Prompt: ### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.

### Raw Source:축구 운동화 남성용 2023 신형 통기성 메쉬 러닝화 경량 편안 학생 캐주얼 스포츠 트렌드 신발
### Product Title:
Generated Text: ### Instruction: Given a raw source for a product, gen

In [ ]:
for text_type, info in timing_info.items():
    if info['count'] > 0:
        average_time = info['total_time'] / info['count']
        print(f"Average time for {text_type}: {average_time:.2f} seconds")

Average time for Product Title: 69.37 seconds
Average time for Detailed Description: 397.50 seconds
Average time for Related Keywords: 65.76 seconds


In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df

,prompt,type,generated_text,total
0,### Instruction: Given a raw source for a prod...,Related Keywords,"지능형 쓰레기통, 자동 쓰레기통, 대용량 쓰레기통, 전동 쓰레기통, 거실 쓰레기통,...",### Instruction: Given a raw source for a prod...
1,### Instruction: Given a raw source for a prod...,Product Title,2023 신형 남성용 경량 통기성 축구 운동화 러닝화 캐주얼 스포츠 신발,### Instruction: Given a raw source for a prod...
2,### Instruction: Given a raw source for a prod...,Related Keywords,"스마트 전자레인지, 다기능 전자레인지, 가정용 전자레인지, 일체형 전자레인지, 고속...",### Instruction: Given a raw source for a prod...
3,### Instruction: Given a raw source for a prod...,Product Title,2023 봄 신상 남성 와이드핏 캐주얼 바지 비즈니스 일상 편안한 팬츠,### Instruction: Given a raw source for a prod...
4,### Instruction: Given a raw source for a prod...,Product Title,2024 신상 여성용 경사금 실크 벨벳 이브닝백 체인 크로스백,### Instruction: Given a raw source for a prod...
5,### Instruction: Given a raw source for a prod...,Detailed Description,애플의 14 는 최신 5 네트워크 기술을 지원하여 전국 어디에서나 빠르고 안정적...,### Instruction: Given a raw source for a prod...
6,### Instruction: Given a raw source for a prod...,Product Title,방수 블루투스 이어폰 무선 스포츠용 소음차단 이어플러그 애플 안드로이드 호환,### Instruction: Given a raw source for a prod...
7,### Instruction: Given a raw source for a prod...,Product Title,바바 농장 자색 피패션프루트 15근 포장 교환용,### Instruction: Given a raw source for a prod...
8,### Instruction: Given a raw source for a prod...,Product Title,2023 봄가을 남성 레트로 오버사이즈 캐주얼 자켓 신상 상의,### Instruction: Given a raw source for a prod...
9,### Instruction: Given a raw source for a prod...,Product Title,블루투스 50 스테레오 무선 이어폰 방수 소음감소 충전 케이스 포함,### Instruction: Given a raw source for a prod...


In [ ]:
results_df.to_csv('/content/drive/MyDrive/ColabNotebooks/generated_text/llama-base-3epoch-results.csv', index=False)
results_df.to_json('/content/drive/MyDrive/ColabNotebooks/generated_text/llama-base-3epoch-results.json', orient='records', lines=True)